In [13]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
import spacy

In [14]:
spacy_nlp = spacy.load('en_core_web_sm')

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_UFC_fighters"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

fighter_links = []
for link in soup.find_all("a"):
    href = link.get("href")
    if href and "/wiki/" in href and "UFC_fighter" in href:
        fighter_links.append("https://en.wikipedia.org" + href)

In [10]:
href_set = set()
for link in soup.find_all("a"):
    href = link.get("href")
    if href and "/wiki/" in href:
        #print(href)
        href_set.add(href.replace("/wiki/", ""))

In [42]:
href_set_lst = list(href_set)
len(href_set_lst)

647

### filter entities extracted from the wiki page, assume any entity that is not a geographic location(GPE) is a UFC fighter; we go this approach because sometimes spacy won't recognize the fighter as person(PERSON) correctly.


In [59]:
href_set_person_lst = []
for entity in href_set_lst:
    if entity[:5]!='https':
        loaded = spacy_nlp(entity.replace("_", " "))
        if loaded.ents:
            #print(loaded.ents)
            #if loaded.ents[0].label_=='PERSON':
            if loaded.ents[0].label_!='GPE':
                url = "https://en.wikipedia.org" + "/wiki/" + entity.replace(" ", "_")
                href_set_person_lst.append(url)

In [60]:
len(href_set_person_lst)

474

In [64]:
with open("data/ufc_wiki_urls_raw.txt", "w") as f:
    for item in href_set_person_lst:
        f.write(f"{item}\n")

### manually remove links that are not UFC fighters

In [ ]:
### e.g. https://en.wikipedia.org/wiki/Strawweight_(MMA), https://en.wikipedia.org/wiki/UFC_290, https://en.wikipedia.org/wiki/Category:Articles_with_hCards